In [21]:
%run init_notebook.py

In [22]:
from src.utils import load_pd_df, get_dt_index

In [173]:
from io import StringIO
import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self

    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio
        sys.stdout = self._stdout

def write_to_txt(output: str, file_name):
    f = open(os.path.join(GRAPHS_DIR, file_name), 'w+')
    f.write(output)
    f.close()
    pass

In [164]:
pi = get_dt_index(load_pd_df('de_inf_prepro.csv'))
pi_e = load_pd_df('infl_expec.feather')
bei = load_pd_df('bei_data.feather')

In [165]:
pi['M'] = pi.index.to_period('M').start_time
pi_e['M'] = pi_e.index.to_period('M').start_time
pi_e.index.name = 'date'
bei['W'] = bei.index.to_period('W').start_time
bei.drop('pi_de_surprise_Y', axis=1, inplace=True)
bei = bei.reset_index().set_index('W')

In [166]:
pi = pi.reset_index().set_index('M')[['pi_de_Y', 'pi_de_surprise_Y', 'date']]
pi_e = pi_e.reset_index().set_index('M')[['pi_e_estimate_mean']]

In [167]:
pi = pi.set_index(pi.date.dt.to_period('W'))
pi.index = pi.index.start_time

In [168]:
pi = pi.join(pi_e)
pi['W'] = pi.date.dt.to_period('W')

In [169]:
pi = pi.set_index('W')
pi.index = pi.index.start_time

In [170]:
df = bei[['date', 'bei']].join(pi.drop('date', axis=1)).copy()

In [180]:
with Capturing() as out:
    print(df.describe().drop('date', axis=1))
write_to_txt("\n".join(out), 'desc.txt')

with Capturing() as out:
    print(df.drop('date', axis=1).cov())
write_to_txt("\n".join(out), 'cov.txt')